<div align="center">
    
# Asset Finance Risk Assessment Model
## Binary Classification
<p><b>Logistic Regression</b></p>
<p><b>Random Forest</b></p>
<p><b>Gradient Boosting</b></p>
<p><b>XGBoost</b></p>

> Author: [Mithamo Beth](https://github.com/Mythamor) 
</div>

## Preprocessing Pipeline (Pipe the final model)

In [1]:
# Import the relevant libraries
import pandas as pd
import numpy as np
import duckdb
import sweetviz as sv

# Visualization libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Set the layout
sns.set_style("darkgrid");

# Set the color palette
sns.set_palette("Set2");

# Set the default figsize
plt.rcParams["figure.figsize"] = (6, 4);

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
new_df = pd.read_csv('../data/new_df.csv', index_col=0)
new_df.head()

,Disbursed_amount,Asset_cost,Ltv,Manufacturer_id,Employment_type,State_id,Aadhar_flag,Voterid_flag,Perform_cns_score,Pri_no_of_accts,Pri_active_accts,New_accts_in_last_six_months,No_of_inquiries,Loan_default,Average_loan_tenure,Credit_history_length_months,Perform_cns_score_description,Age
0,50578,58400,89.55,45,Salaried,6,1,0,0,0,0,0,0,0,0.00,0,No Bureau History Available,34
1,47145,65550,73.23,45,Self employed,6,1,0,598,1,1,0,0,1,1.92,23,I-Medium Risk,33
2,53278,61360,89.63,45,Self employed,6,1,0,0,0,0,0,0,0,0.00,0,No Bureau History Available,32
3,57513,66113,88.48,45,Self employed,6,1,0,305,3,0,0,1,1,0.67,15,L-Very High Risk,24
4,52378,60300,88.39,45,Self employed,6,1,0,0,0,0,0,1,1,0.00,0,No Bureau History Available,40


In [3]:
# Import preprocessing libraries
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [4]:
# Split the target and predictor variables
X = new_df.drop(['Loan_default'], axis=1)
y = new_df['Loan_default']

# Perfrom a train test split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print('X_train:', X_train.shape, '\n X_test:',X_test.shape)
print('y_train:', y_train.shape, '\n y_test:',y_test.shape)

X_train: (163207, 17) 
 X_test: (69947, 17)
y_train: (163207,) 
 y_test: (69947,)


In [5]:
y_train.head()

81759     1
115977    1
51043     0
18380     0
192570    1
Name: Loan_default, dtype: int64

In [6]:
X_train.head()

,Disbursed_amount,Asset_cost,Ltv,Manufacturer_id,Employment_type,State_id,Aadhar_flag,Voterid_flag,Perform_cns_score,Pri_no_of_accts,Pri_active_accts,New_accts_in_last_six_months,No_of_inquiries,Average_loan_tenure,Credit_history_length_months,Perform_cns_score_description,Age
81759,56013,67722,85.64,86,Self employed,4,1,0,738,1,1,1,0,0.33,4,C-Very Low Risk,39
115977,53278,68455,79.76,86,Self employed,4,1,0,694,2,1,0,0,0.58,7,E-Low Risk,23
51043,27929,62459,48.03,45,Salaried,1,1,0,0,0,0,0,0,0.00,0,No Bureau History Available,39
18380,35275,64929,56.99,86,Self employed,3,1,0,690,10,2,2,0,1.33,37,E-Low Risk,55
192570,67582,81193,84.98,86,Missing,8,1,0,681,2,1,0,1,0.83,11,E-Low Risk,18


In [7]:
# Ensure these are correct numeric and categorical columns
numeric_cols = X_train.select_dtypes(include=['int64', 'float64'])
category_cols = X_train.select_dtypes(include=['object', 'category'])
nums =  numeric_cols.columns
cats = category_cols.columns
print(f'Numeric Columns:\n {nums}\n\n')
print(f'Categorical Columns\n :{cats}\n\n')

Numeric Columns:
 Index(['Disbursed_amount', 'Asset_cost', 'Ltv', 'Manufacturer_id', 'State_id',
       'Aadhar_flag', 'Voterid_flag', 'Perform_cns_score', 'Pri_no_of_accts',
       'Pri_active_accts', 'New_accts_in_last_six_months',
       'Average_loan_tenure', 'Credit_history_length_months', 'Age'],
      dtype='object')


Categorical Columns
 :Index(['Employment_type', 'No_of_inquiries', 'Perform_cns_score_description'], dtype='object')




In [8]:
# Confirm numeric columns
nums =  numeric_cols.columns
nums

Index(['Disbursed_amount', 'Asset_cost', 'Ltv', 'Manufacturer_id', 'State_id',
       'Aadhar_flag', 'Voterid_flag', 'Perform_cns_score', 'Pri_no_of_accts',
       'Pri_active_accts', 'New_accts_in_last_six_months',
       'Average_loan_tenure', 'Credit_history_length_months', 'Age'],
      dtype='object')

In [9]:
# Confirm categorical columns
cats = category_cols.columns
cats

Index(['Employment_type', 'No_of_inquiries', 'Perform_cns_score_description'], dtype='object')

In [10]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [11]:
# Data Preprocessing pipeline
# Numerical pipeline
numerical_pipeline = Pipeline(steps=[
    ('ss', StandardScaler())
])

# Categorical pipeline
categorical_pipeline = Pipeline(steps=[
    ('ohe', OneHotEncoder())
])

# Combined preprocessed data
trans = ColumnTransformer(transformers=[
    ('numerical', numerical_pipeline, nums),
    ('categorical', categorical_pipeline, cats)
], remainder="passthrough")

# Preprocesing pipeline
preprocessing_pipe = Pipeline(steps=[
                    ('col_trans', trans)
                    ])


In [16]:
# FIT AND TRANSFORM the training data
print("Applying preprocessing pipeline...")
X_train_preprocessed = preprocessing_pipe.fit_transform(X_train)

# Check the shape after preprocessing
print(f"Shape after preprocessing: {X_train_preprocessed.shape}")


Applying preprocessing pipeline...
Shape after preprocessing: (163207, 41)


# Classification Modeling

If capturing all actual loan defaults is more important, prioritize recall. Recall (also known as sensitivity or true positive rate) measures the proportion of correctly predicted positive instances (loan defaults) out of all actual positive instances. Recall focuses on the ability of the model to capture all positive instances, which is crucial in identifying all actual loan defaults and minimizing false negatives.

In [12]:
# Classification modeling libraries
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, accuracy_score, recall_score\
,roc_curve, auc, precision_score, RocCurveDisplay

In [13]:
from sklearn.metrics import (accuracy_score, recall_score, precision_score,
                             f1_score, confusion_matrix, roc_curve, auc, 
                             ConfusionMatrixDisplay, RocCurveDisplay)
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Function for model evaluation
def eval_model(model, X_test, y_test, n_classes=None):
    preds = model.predict(X_test)
    probs = model.predict_proba(X_test)
    
    accuracy = accuracy_score(y_test, preds)
    recall = recall_score(y_test, preds, average='weighted')
    precision = precision_score(y_test, preds, average='weighted')
    f1 = f1_score(y_test, preds, average='weighted')
    con_max = confusion_matrix(y_test, preds)
    
    if n_classes is None:
        n_classes = model.classes_
    
    # Display the confusion matrix
    plt.figure(figsize=(8, 6))
    disp = ConfusionMatrixDisplay(con_max, display_labels=n_classes)
    disp.plot(cmap=plt.cm.magma)
    plt.title("Confusion Matrix")
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

    print("\n")
    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1: ", f1)
    
    # ROC Curve for multiclass (One-vs-Rest)
    plt.figure(figsize=(10, 8))
    for i in range(len(n_classes)):
        fpr, tpr, _ = roc_curve(y_test, probs[:, i], pos_label=i)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'ROC curve for {n_classes[i]} (area = {roc_auc:.2f})')

    # Plot the chance level line
    plt.plot([0, 1], [0, 1], color='darkorange', linestyle='--')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.show()

    results_df = pd.DataFrame()
    results_df['true_class'] = y_test
    results_df['predicted_class'] = preds
    results_df['predicted_probabilities'] = list(probs)

    # Plot the distribution of probabilities for the estimated classes
    for i in range(len(n_classes)):
        sns.kdeplot(results_df[results_df['true_class'] == n_classes[i]]['predicted_probabilities'][:, i],
                    label=f"{n_classes[i]}", fill=True)

    plt.title('Probability Distributions for Each Class')
    plt.xlabel('Predicted Probability')
    plt.ylabel('Density')
    plt.legend()
    plt.show()
    
    # See the true class versus predicted class as a percentage
    print(results_df.groupby('true_class')['predicted_class'].value_counts(normalize=True))

### Class Imbalance

In [17]:
# Class imbalance 
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample

# Previous original class distribution
print('Original class distribution: \n')
print(new_df['Loan_default'].value_counts())

smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_preprocessed, y_train) 
# Preview synthetic sample class distribution

print('-----------------------------------------')
print('Synthetic sample class distribution: \n')
print(pd.Series(y_train_resampled).value_counts()) 

Original class distribution: 

Loan_default
0    182543
1     50611
Name: count, dtype: int64
-----------------------------------------
Synthetic sample class distribution: 

Loan_default
1    127787
0    127787
Name: count, dtype: int64


In [19]:
# Now let's compare a few different ratios of minority class to majority class
ratios = [0.5, 0.7, 1]
names = ['0.5','0.7','even']
colors = sns.color_palette('Set2')

plt.figure(figsize=(10, 8))

for n, ratio in enumerate(ratios):
    # Fit a model
    smote = SMOTE(sampling_strategy=ratio)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_preprocessed, y_train) 
    logreg = LogisticRegression(fit_intercept=False, C=1e20, solver ='lbfgs')
    model_log = logreg.fit(X_train_resampled, y_train_resampled)
    print(model_log)

    # Predict
    y_hat_test = logreg.predict(X_test)

    y_score = logreg.decision_function(X_test)

    fpr, tpr, thresholds = roc_curve(y_test, y_score)
    
    print('AUC for {}: {}'.format(names[n], auc(fpr, tpr)))
    print('-------------------------------------------------------------------------------------')
    lw = 2
    plt.plot(fpr, tpr, color=colors[n],
             lw=lw, label='ROC curve {}'.format(names[n]))

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

plt.yticks([i/20.0 for i in range(21)])
plt.xticks([i/20.0 for i in range(21)])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

LogisticRegression(C=1e+20, fit_intercept=False)


ValueError: could not convert string to float: 'Self employed'

The best performing model is when our samples are balanced at  0.5. The smote model is not performing any better than the ordinary regression model though.

## Target Variable

In [19]:
y_train.value_counts(normalize=True)

Loan_default
0    0.782975
1    0.217025
Name: proportion, dtype: float64

## Binary Classification Models

### Logistic Regression

In [20]:
smote = SMOTE(sampling_strategy='minority')
X_train_SMOTE, y_train_SMOTE = smote.fit_resample(X_train,y_train)
# Visualize the proportion of borrowers
show_loan_distrib(y_train_SMOTE)

ValueError: could not convert string to float: 'Self employed'

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Instantiate and fit the model
log_reg = LogisticRegression(solver='liblinear')
log_reg.fit(X_train, y_train)

# Make predictions
preds = log_reg.predict(X_test)

# Score the train and test data for model performance accuracy
print(f'Training Accuracy: {log_reg.score(X_train, y_train)}')
print(f'Testing Accuracy : {log_reg.score(X_test, y_test)}')

# Evaluate the model
print(classification_report(y_test, preds))

### Random Forest Classifier

In [21]:
from sklearn.ensemble import RandomForestClassifier

# Instantiate and fit the model
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)

# Make predictions
preds = rfc.predict(X_test)

# Score the train and test data for model performance accuracy
print(f'Training Accuracy: {rfc.score(X_train, y_train)}')
print(f'Testing Accuracy : {rfc.score(X_test, y_test)}')

# Evaluate the model
print(classification_report(y_test, preds))


ValueError: could not convert string to float: 'Self employed'

In [ ]:
# Hypertune Random Forest classifier
rfc_1 = RandomForestClassifier(
                        n_estimators=10,
                        criterion='gini',
                        min_samples_split=5,
                        min_samples_leaf=3,
                        max_depth=10, 
                        class_weight= "balanced")

# Fit the clasifier
rfc_1.fit(X_train, y_train) 

# Score the train and test data for model performance accuracy
print(f'Training Accuracy: {rfc_1.score(X_train, y_train)}')
print(f'Testing Accuracy : {rfc_1.score(X_test, y_test)}')

# Evaluate the model
print(classification_report(y_test, preds))

### Gradient Boosting

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

# Instantiate and fit the model
gbc = GradientBoostingClassifier(n_estimators=100, random_state=42)
gbc.fit(X_train, y_train)

# Make predictions
preds = gbc.predict(X_test)

# Score the train and test data for model performance accuracy
print(f'Training Accuracy: {gbc.score(X_train, y_train)}')
print(f'Testing Accuracy : {gbc.score(X_test, y_test)}')

# Evaluate the model
print(classification_report(y_test, preds))


ValueError: could not convert string to float: 'Self employed'

### XGBoost

####  Encode the target variable

In [23]:
# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit the label encoder and transform y_train and y_test
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


In [24]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Instantiate and fit the model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train, y_train)

x_train = xgb_model.predict(X_train)
preds = xgb_model.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(y_train, x_train)
test_accuracy = accuracy_score(y_test, preds)

print(f'Training Accuracy: {train_accuracy}')
print(f'Testing Accuracy : {test_accuracy}')

# Evaluate the model
print(classification_report(y_test, preds))


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Employment_type: object, No_of_inquiries: object, Perform_cns_score_description: object

In [ ]:
from imblearn.pipeline import Pipeline

rf_clf = RandomForestClassifier(criterion='gini', bootstrap=True, random_state=100)
smote_sampler = SMOTE(random_state=9)
pipeline = Pipeline(steps = [['smote', smote_sampler],
                             ['classifier', rf_clf]])
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)

# Score the train and test data for model performance accuracy
print(f'Training Accuracy: {rf_clf.score(X_train, y_train)}')
print(f'Testing Accuracy : {rf_clf.score(X_test, y_test)}')

# Evaluate the model
print(classification_report(y_test, preds))


In [ ]:
eval_model(rf_clf, X_test, y_test)

In [ ]:
# RandomForest GridSearchCV
rf_param_grid = {
    'n_estimators': [10, 30, 100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 6, 10],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [3, 6]
}

# Instatiate the clasifier
rf_clf = RandomForestClassifier()

# Grid Search
rf_grid_search = GridSearchCV(rf_clf, rf_param_grid, cv=3)

# Fit the model
rf_grid_search.fit(X_train, y_train)

print(f"Training Accuracy: {rf_grid_search.best_score_ :.2%}")
print("")
print(f"Optimal Parameters: {rf_grid_search.best_params_}")

eval_model(rf_grid_search, X_test, y_test)

In [ ]:
# Feature Importance
feature_importance = rfc.feature_importances_

# Create a DataFrame to associate feature names with coefficients
coefficients_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importance})

coefficients_df = coefficients_df.sort_values(by='Importance', ascending=False)

# Print the DataFrame with ranked coefficients
selected = coefficients_df['Feature'].iloc[:30].to_list()
selected

In [25]:
# The logloss of the RFC
from sklearn.metrics import log_loss
log_loss(y_test, rfc.predict_proba(X_test)[:, 1])

AttributeError: 'RandomForestClassifier' object has no attribute 'estimators_'

> The rfc model, has still failed to atch up to the gostic model. Let's explore more ensemble methods before concluding.

## AdaBoost Classifier & GradientBoosting Classifier

In [26]:
# Instantiate an AdaBoostClassifier
adaboost_clf =AdaBoostClassifier(n_estimators=100, learning_rate=2, random_state=0)

# Fit AdaBoostClassifier
adaboost_clf.fit(X_train_resampled, y_train_resampled)

# AdaBoost model predictions
adaboost_train_preds = adaboost_clf.predict(X_train_resampled)
adaboost_test_preds = adaboost_clf.predict(X_test)

def display_acc_and_f1_score(true, preds, model_name):
    acc = accuracy_score(true, preds)
    f1 = f1_score(true, preds)
    print("Model: {}".format(model_name))
    print("Accuracy: {}".format(acc))
    print("F1-Score: {}".format(f1))
    

# Adaboost Model Evaluation   
print("Training Metrics")
display_acc_and_f1_score(y_train_resampled, adaboost_train_preds, model_name='Train AdaBoost')
print("")

print("Testing Metrics")
display_acc_and_f1_score(y_test, adaboost_test_preds, model_name=' Test AdaBoost')
print("")

eval_model(adaboost_clf, X_test, y_test)

NameError: name 'X_train_resampled' is not defined

> Adaboosting performed very poorly on the test data and classifying the non_default loans

In [27]:
# Instantiate an GradientBoostingClassifier
gbt_clf = GradientBoostingClassifier(learning_rate=0.1,
                                    n_estimators=10,
                                    random_state=42,
                                    min_samples_split=3,
                                    min_samples_leaf=2,
                                    max_depth = 5,
                                    max_features=5,
                                    max_leaf_nodes=5)
# Fit GradientBoostingClassifier
gbt_clf.fit(X_train_resampled, y_train_resampled)

# GradientBoosting model predictions
gbt_clf_train_preds = gbt_clf.predict(X_train_resampled)
gbt_clf_test_preds = gbt_clf.predict(X_test)

# Gradient Boosting Evaluation
display_acc_and_f1_score(y_train_resampled, gbt_clf_train_preds, model_name='Train Gradient Boosted Trees')
print("")
display_acc_and_f1_score(y_test, gbt_clf_test_preds, model_name=' Test Gradient Boosted Trees')
print()
eval_model(gbt_clf, X_test, y_test)

NameError: name 'X_train_resampled' is not defined

> This model may be very good at predicting loan deafults, but it fails miserably on all other metrics.
> 92% of our non_defaults are classified as default. This could lead to business loss, due to the poor performance of the model, to accurately classify non_defaults.

In [28]:
# Gradient Boosting GridSearch
from sklearn.metrics import make_scorer
# Instantiate the model
gbc = GradientBoostingClassifier()

# Parameter Grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.2],
    'max_depth': [3, 6, 9],
    'min_samples_leaf': [1, 3, 6],
    'min_samples_split': [2, 3, 6],
    'max_features': [20, 30, 50],
    'subsample': [0.6, 0.8, 1.0]
}

scorer = make_scorer(roc_auc)
# GridSearhCV
grid_gbc = GridSearchCV(gbc, param_grid, scoring=scorer, cv=3, n_jobs=1)
grid_gbc.fit(X_train_resampled, y_train_resampled)

best_parameters = grid_gbc.best_params_

print('Grid Search found the following optimal parameters: ')
for param_name in sorted(best_parameters.keys()):
    print('%s: %r' % (param_name, best_parameters[param_name]))

training_probs = grid_gbc.predict_proba(X_train_resampled)[:, 1]
test_probs = grid_gbc.predict_proba(X_test)[:, 1]
training_auc = roc_auc_score(y_train_resampled, training_probs)
test_auc = roc_auc_score(y_test, test_probs)

print('')
print('Training AUC-ROC: {:.4}'.format(training_auc))
print('Validation AUC-ROC: {:.4}'.format(test_auc))

eval_model(grid_gbc, X_test, y_test)

NameError: name 'roc_auc' is not defined

## XGBoost Classifier

In [29]:
# Import XGBoosting
from xgboost import XGBClassifier

In [30]:
learning_rate_range = np.arange(0.01, 1, 0.05)
test_XG = []
train_XG = []
for lr in learning_rate_range:
    xgb = XGBClassifier(eta=lr)
    xgb.fit(X_train_resampled, y_train_resampled)
    train_XG.append(xgb.score(X_train_resampled, y_train_resampled))
    train_XG.append(xgb.score(X_test, y_test))

NameError: name 'X_train_resampled' is not defined

In [31]:
# Instantiate the model
xgb = XGBClassifier()

# Parameter Grid
param_grid = {
    'learning_rate': [0.1, 0.2],
    'max_depth': [6],
    'min_child_weight': [1, 2],
    'subsample': [0.5, 0.7],
    'n_estimators': [100],
}

# GridSearhCV
grid_xgb = GridSearchCV(xgb, param_grid, scoring='accuracy', cv=3, n_jobs=1)
grid_xgb.fit(X_train, y_train)

best_parameters = grid_xgb.best_params_
-----------------------------
print('Grid Search found the following optimal parameters: ')
for param_name in sorted(best_parameters.keys()):
    print(f"{param_name}: {best_parameters[param_name]}")

training_preds = grid_xgb.predict(X_train)
test_preds = grid_xgb.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
test_accuracy = accuracy_score(y_test, test_preds)

print('')
print('Training Accuracy: {:.4}%'.format(training_accuracy * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy * 100))
      
eval_model(grid_xgb, X_test, y_test)

SyntaxError: invalid syntax (3159888617.py, line 18)

## Model Deployment

For now we will settle on the 2nd logistic model, that had a fair balance of all the metrics, the highest recall without compromising on the other metrics.

* model.pkl (a pickled model file)
* main.py (a Python file defining the function)
* requirements.txt (a requirements file)

In [32]:
# Create a copy of the dataframe
df_copy = df.copy()

# Split the target and predictor variables
X2 = df_copy.drop(['Loan_default'], axis=1)
y2 = df_copy['Loan_default']

NameError: name 'df' is not defined

In [33]:
# Perfrom a train test split
X_train2 ,X_test2 ,y_train2 ,y_test2 = train_test_split(X, y, test_size=0.25, random_state=42)

print('X_train:', X_train2.shape, '\n X_test:',X_test2.shape)

X_train: (174865, 17) 
 X_test: (58289, 17)


In [34]:
# Data Preprocessing pipeline
# Numerical pipeline
numerical_pipeline = Pipeline(steps=[
    ('ss', MinMaxScaler())
])

# Categorical pipeline
categorical_pipeline = Pipeline(steps=[
    ('ohe', OneHotEncoder(drop='first',
                         sparse=False))
])

# Combined preprocessed data
trans = ColumnTransformer(transformers=[
    ('numerical', numerical_pipeline, nums),
    ('categorical', categorical_pipeline, cats)
], remainder="passthrough")

# Logistic model
log_model2 = LogisticRegression(fit_intercept=True, 
                               C=1e12, 
                               solver='liblinear',
                               max_iter=300, 
                               penalty='l1',
                               class_weight="balanced")
# Pipe the model
log_pipe = Pipeline(steps=[
        ("col_trans", trans),
        ("log_model2", log_model2)])

# Fit the training data
log_pipe.fit(X_train2, y_train2)
    
# Score the train and test data for model performance accuracy
print(f'Training Accuracy: {log_pipe.score(X_train2, y_train2)}')
print(f'Testing Accuracy : {log_pipe.score(X_test2, y_test2)}')

# Evaluate the model
eval_model(log_pipe, X_test2, y_test2)

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

In [35]:
# Assuming you have 'X_train' as your feature matrix and 'nums' and 'cats' as your lists of numerical and categorical columns

# Fit and transform the data using the preprocessing_pipe
X_preprocessed = preprocessing_pipe.fit_transform(X_train2)

# Get the column names from the transformed data
transformed_feature_names = []

# Get numerical feature names from the standard scaler
numerical_feature_names = X_train2[nums].columns
transformed_feature_names.extend(numerical_feature_names)

# Get one-hot encoded categorical feature names from the OneHotEncoder
categorical_encoder = preprocessing_pipe.named_steps['col_trans'].transformers_[1][1].named_steps['ohe']
categorical_feature_names = categorical_encoder.get_feature_names_out(X_train2[cats].columns)
transformed_feature_names.extend(categorical_feature_names)

# If you have remaining columns that were not transformed, add them to the list
remaining_columns = [col for col in X_train2.columns if col not in numerical_feature_names and col not in categorical_feature_names]
transformed_feature_names.extend(remaining_columns)

# Print the preprocessed column names
print(transformed_feature_names)

['Disbursed_amount', 'Asset_cost', 'Ltv', 'Manufacturer_id', 'State_id', 'Aadhar_flag', 'Voterid_flag', 'Perform_cns_score', 'Pri_no_of_accts', 'Pri_active_accts', 'New_accts_in_last_six_months', 'Average_loan_tenure', 'Credit_history_length_months', 'Age', 'Employment_type_Missing', 'Employment_type_Salaried', 'Employment_type_Self employed', 'No_of_inquiries_0', 'No_of_inquiries_1', 'No_of_inquiries_2', 'No_of_inquiries_3+', 'Perform_cns_score_description_A-Very Low Risk', 'Perform_cns_score_description_B-Very Low Risk', 'Perform_cns_score_description_C-Very Low Risk', 'Perform_cns_score_description_D-Very Low Risk', 'Perform_cns_score_description_E-Low Risk', 'Perform_cns_score_description_F-Low Risk', 'Perform_cns_score_description_G-Low Risk', 'Perform_cns_score_description_H-Medium Risk', 'Perform_cns_score_description_I-Medium Risk', 'Perform_cns_score_description_J-High Risk', 'Perform_cns_score_description_K-High Risk', 'Perform_cns_score_description_L-Very High Risk', 'Perfor

In [36]:
# Verify the prediction with an example
print(X_train2.iloc[0])
print(y_train2.iloc[0])

Disbursed_amount                                       64882
Asset_cost                                             75217
Ltv                                                    89.08
Manufacturer_id                                           45
Employment_type                                Self employed
State_id                                                   3
Aadhar_flag                                                1
Voterid_flag                                               0
Perform_cns_score                                          0
Pri_no_of_accts                                            0
Pri_active_accts                                           0
New_accts_in_last_six_months                               0
No_of_inquiries                                            0
Average_loan_tenure                                      0.0
Credit_history_length_months                             0.0
Perform_cns_score_description    No Bureau History Available
Age                     

In [37]:
# Our model is accurate in this instance
log_pipe.predict(X_train2)[0]

NameError: name 'log_pipe' is not defined

In [38]:
# Requirements.txt
import sklearn
import joblib
print(sklearn.__version__)
print(joblib.__version__)

1.5.1
1.4.2


In [39]:
with open("model.pkl", "wb") as f:
    joblib.dump(log_pipe, f)

NameError: name 'log_pipe' is not defined

In [40]:
# Assuming 'log_reg' is your fitted Logistic Regression model
class CallableModel:
    def __init__(self, model):
        self.model = model

    def __call__(self, X):
        return self.model.predict_proba(X)

# Wrap your logistic regression model
callable_log_reg = CallableModel(log_reg)

# Create the SHAP explainer
explainer = shap.KernelExplainer(callable_log_reg, X_train)  # Use X_train for background data

# Calculate SHAP values
shap_values = explainer.shap_values(X_test)

# Visualize SHAP values for the first instance in the test set
shap.summary_plot(shap_values, X_test)

NameError: name 'log_reg' is not defined

### Model function

In [ ]:
import numpy as np

def predict_credit_limit_and_score(user_data):
    """
    Predicts both the credit limit (regression) and the credit score category (classification).
    
    Parameters:
    - user_data: A dictionary or Pandas Series containing the user's input data for the prediction.
    
    Returns:
    - result: A dictionary with predicted credit limit and credit score category.
    """
    
    # 1. Preprocess the input data as required by your models
    # Assume you have a function `preprocess_input` that applies necessary transformations
    processed_data = preprocess_input(user_data)
    
    # 2. Predict the credit limit using the trained regressor model
    credit_limit = credit_limit_model.predict(processed_data)
    
    # 3. Predict the credit score category using the trained classifier model
    credit_score_class = credit_score_model.predict(processed_data)
    
    # 4. Map the numeric class prediction to a credit score category label
    score_mapping = {0: 'Very Low', 1: 'Low', 2: 'Medium', 3: 'High', 4: 'Very High'}
    credit_score_category = score_mapping.get(credit_score_class[0], 'Unknown')
    
    # 5. Return both the credit limit and the score category as the result
    result = {
        'credit_limit': credit_limit[0],  # Assuming the model returns an array
        'credit_score_category': credit_score_category
    }
    
    return result


In [ ]:
data_1 = [200000, 150000, 75, 6,
       5, 0, 50000,
       120000, 0,
       0, 18,
       0, 35, 7,
       0, 1,
       1, 0, 0,
       0, 1,
       1, 0,
       0, 0,
       0,0,
       1,1,0,
       0,0,0,
       0,0,0,
       1,0,0,0,0,
       0,0,0,0,0,
       0,0,0,0,
       0,0,0,0,
       0,0,1,1]
# List of values for prediction
data_1 = [800000, 750000, 50, 2, 5, 5, 50000, 150000, 0, 0, 18, 0, 45, 7, 0, 1, 1, 0, 0, 0,
          0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# Create a DataFrame using the provided values
data_1_df = pd.DataFrame([data_1], columns=X_train.columns)

# Make the prediction using the trained RFC model
prediction = rfc.predict(data_1_df)

print("Predicted class:", prediction)

In [338]:
# Convert the Pandas Series to a NumPy array and then reshape it
data_point = X_test.iloc[0].values.reshape(1, -1)

# Make the prediction
prediction = rfc.predict(data_point)

print("Predicted class:", prediction)

Predicted class: [0]


## Conclusion